# Intro

Very basic CNN for recognizing hand-drawn images of various objects, to which Gaussian noise was added. <br>
One class of objects contain only noise. <br>
The dataset is no longer available but the CNN results can still be seen below <br><br>
tl;dr ~60% accuracy (disappointing). Noise reduction & feature extraction would probably help

In [0]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import numpy as np 
import pickle
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras import models, layers
import os
import pandas as pd

## 1. Data loading & preprocessing

### Set seed for reproducibility

In [0]:
SEED = 123
np.random.seed(SEED)

### Load data

In [0]:
DIR = "input" # Location of input data

In [0]:
X_train = np.load(os.path.join(DIR, "train_images.npy"), encoding='latin1')
train_labels = np.genfromtxt(os.path.join(DIR, "train_labels.csv"), names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'S15')])
X_test = np.load(os.path.join(DIR, "test_images.npy"), encoding = 'latin1')

X_train = np.array(tuple(x[1] for x in X_train))
X_test = np.array(tuple(x[1] for x in X_test))

### Numerically-encode image labels

In [0]:
# Numerical encoding
y = train_labels[:]['Category']
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

### Normalize data

In [0]:
# converts images to greyscale
X_train /= X_train.max()
X_test /= X_test.max()

### Split into training and validation sets

In [0]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train,
                                                      y,
                                                      random_state=SEED,
                                                      stratify=y/len(y)
                                                     )

### Reshape data for keras

In [0]:
# Reshape to 4D tensor (last dimension is nb. of channels)
X_train = X_train.reshape(X_train.shape[0], 100, 100, 1)
X_valid = X_valid.reshape(X_valid.shape[0], 100, 100, 1)
X_test = X_test.reshape(X_test.shape[0], 100, 100, 1)

# One-hot encode labels
n_classes = len(np.unique(y))
y_train = to_categorical(y_train, n_classes)
y_valid = to_categorical(y_valid, n_classes)

### Augment data through transformations

In [0]:
# Training batch size
BATCH_SIZE = 32

In [0]:
# Load datagen if already saved
path_to_datagen = os.path.join(DIR, "datagen")

try:
    with open(path_to_datagen, "rb") as f:
        datagen = pickle.load(f)
        
except FileNotFoundError:
    # Create datagen
    datagen = ImageDataGenerator(rotation_range=30, # rotate images by [0,30] deg.
                                horizontal_flip=True,
                                )
    datagen.fit(X_train,
               seed=SEED
               )

In [0]:
# Save datagen to file
path_to_datagen = os.path.join(DIR, "datagen")
with open(path_to_datagen, "wb") as f:
    pickle.dump(datagen, f)

In [0]:
# Create a transformed generator of X_train
X_train_transformed = datagen.flow(X_train,
                y_train,
                batch_size=BATCH_SIZE,
                seed=SEED
               )

## 2. Model

### CNN

In [12]:
# Load CNN if already saved
path_to_cnn = os.path.join(DIR, "cnn.h5")
if os.path.isfile(path_to_cnn):
        cnn = models.load_model(path_to_cnn)

else:
    
    # Create CNN
    # Architecture: ((2D convolution)*2->MaxPool->Dropout regularizer)*2->Flatten->Dropout->Probability vector

    cnn = models.Sequential()

    cnn.add(layers.Conv2D(filters=32,
                         kernel_size=(3, 3),
                         activation="relu",
                         input_shape=(100, 100, 1),
                         padding="same",
                         ))

    cnn.add(layers.Conv2D(filters=64,
                         kernel_size=(3, 3),
                         activation="relu",
                         padding="same",
                         ))

    cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
    
    cnn.add(layers.Dropout(0.25))

    cnn.add(layers.Conv2D(filters=64,
                         kernel_size=(3, 3),
                         activation="relu",
                         padding="same",
                         ))

    cnn.add(layers.Conv2D(filters=64,
                         kernel_size=(3, 3),
                         activation="relu",
                         padding="same",
                         ))
    
    cnn.add(layers.MaxPooling2D(pool_size=(2, 2)))
    cnn.add(layers.Dropout(0.25))
    
    cnn.add(layers.Flatten())

    cnn.add(layers.Dense(256, activation="relu"))

    cnn.add(layers.Dropout(0.5))

    cnn.add(layers.Dense(n_classes, activation="softmax"))


    cnn.compile(optimizer=keras.optimizers.Adadelta(),
               loss="categorical_crossentropy",
               metrics=["accuracy"])
    # Fit CNN
    losses = cnn.fit_generator(X_train_transformed,
                     epochs=75,
                     steps_per_epoch=X_train.shape[0] // BATCH_SIZE,
                     verbose=2, 
                     validation_data=(X_valid, y_valid),
                     )

Epoch 1/75
 - 26s - loss: 3.4000 - acc: 0.0466 - val_loss: 3.3884 - val_acc: 0.0572
Epoch 2/75
 - 22s - loss: 3.3901 - acc: 0.0523 - val_loss: 3.3792 - val_acc: 0.0572
Epoch 3/75
 - 22s - loss: 3.3823 - acc: 0.0559 - val_loss: 3.3660 - val_acc: 0.0564
Epoch 4/75
 - 22s - loss: 3.3435 - acc: 0.0665 - val_loss: 3.2844 - val_acc: 0.0656
Epoch 5/75
 - 22s - loss: 3.2781 - acc: 0.0815 - val_loss: 3.3462 - val_acc: 0.0668
Epoch 6/75
 - 22s - loss: 3.2233 - acc: 0.0904 - val_loss: 3.1801 - val_acc: 0.1020
Epoch 7/75
 - 22s - loss: 3.1760 - acc: 0.0967 - val_loss: 3.1188 - val_acc: 0.1208
Epoch 8/75
 - 22s - loss: 3.1446 - acc: 0.1009 - val_loss: 3.0972 - val_acc: 0.1076
Epoch 9/75
 - 22s - loss: 3.1117 - acc: 0.1110 - val_loss: 3.1106 - val_acc: 0.1064
Epoch 10/75
 - 22s - loss: 3.0848 - acc: 0.1131 - val_loss: 3.0844 - val_acc: 0.1044
Epoch 11/75
 - 22s - loss: 3.0702 - acc: 0.1222 - val_loss: 3.0531 - val_acc: 0.1324
Epoch 12/75
 - 22s - loss: 3.0029 - acc: 0.1491 - val_loss: 2.8860 - val_a

In [0]:
cnn.save(os.path.join(DIR, "cnn.h5"))

### Save test predicitions

In [0]:
import pandas as pd
predictions = cnn.predict(X_test).argmax(axis=1)
predictions_string = le.inverse_transform(predictions).astype('U15')
df = pd.DataFrame(predictions_string)
df.index.name = "Id"
df.to_csv(os.path.join(DIR, "submission.csv"), header = ["Category"])